In [ ]:
import os
import re
import sys
import nltk
import random
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from nltk.tokenize import PunktSentenceTokenizer, word_tokenize

In [ ]:
#Indonesian SMS Preprocessing
def convertTackyText(text):
    words = word_tokenize(text)
    new_string = ''
    for msg in words:
        new_word = ''
        alpha_flag = False
        digit_flag = False
        for c in msg:
            if c.isalpha():
                alpha_flag = True
            elif c.isdigit():
                digit_flag = True
        
        if alpha_flag and digit_flag:
            msg = msg.lower()
            if msg[-4:] != 'ribu' and msg[-3:] != 'rbu' and msg[-2:] != 'rb':
                for c in msg:
                    if c == '1':
                        c = 'i'
                    elif c == '2':
                        c = 's'
                    elif c == '3':
                        c = 'e'
                    elif c == '4':
                        c = 'a'
                    elif c == '5':
                        c = 's'
                    elif c == '6':
                        c = 'g'
                    elif c == '7':
                        c = 't'
                    elif c == '8':
                        c = 'b'
                    elif c == '9':
                        c = 'g'
                    elif c == '0':
                        c = 'o'
                    new_word = new_word + c
        
        if new_word != '':
            new_string = new_string + new_word + ' '
        else:
            new_string = new_string + msg + ' '

    return new_string

def preproccess_text(text_messages):
    # change words to lower case
    processed = text_messages.lower()

    # Replace email addresses with 'emailaddress'
    processed = re.sub(r'^.+@[^\.].*\.[a-z]{2,}$', ' emailaddress ', processed)
        
    # Replace phone numbers (formats include paranthesis, spaces, no spaces, dashes) with 'phonenumber'
    processed = re.sub(r'(\()?(\+62|62|0)(\d{2,3})?\)?[ .-]?\d{2,4}[ .-]?\d{2,4}[ .-]?\d{2,4}', ' phonenumber ', processed)

    # Replace URLs with 'webaddress'
    processed = re.sub(r'[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)', ' webaddress ', processed)
    processed = processed.replace('http', '')
    processed = processed.replace('https', '')
    
    # Replace money symbols with 'moneysymbol' (£ can by typed with ALT key + 156)
    processed = re.sub(r'£|\$', 'moneysymbol ', processed)
    processed = processed.replace(' rp.', ' moneysymbol ')
    processed = processed.replace(' rp', ' moneysymbol ')
        
    # Replace numbers with 'number'
    processed = re.sub(r'\d+(\.\d+)?', ' number ', processed)

    # Remove punctuation
    processed = re.sub(r'[.,\/#!%\^&\*;:+{}=\-_`~()?]', ' ', processed)

    # Replace whitespace between terms with a single space
    processed = re.sub(r'\s+', ' ', processed)

    # Remove leading and trailing whitespace
    processed = re.sub(r'^\s+|\s+?$', '', processed)
    return processed

def preproccess_df(text_messages):
    # change words to lower case
    processed = text_messages.str.lower()

    # Replace email addresses with 'emailaddress'
    processed = processed.str.replace(r'^.+@[^\.].*\.[a-z]{2,}$', ' emailaddress ')
        
    # Replace phone numbers (formats include paranthesis, spaces, no spaces, dashes) with 'phonenumber'
    processed = processed.str.replace(r'(\()?(\+62|62|0)(\d{2,3})?\)?[ .-]?\d{2,4}[ .-]?\d{2,4}[ .-]?\d{2,4}', ' phonenumber' )

    # Replace URLs with 'webaddress'
    processed = processed.str.replace(r'[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)', ' webaddress ')
    processed = processed.str.replace('http', '')
    processed = processed.str.replace('https', '')
    
    # Replace money symbols with 'moneysymbol' (£ can by typed with ALT key + 156)
    processed = processed.str.replace(r'£|\$', ' moneysymbol ')
    processed = processed.str.replace(' rp.', ' moneysymbol ')
    processed = processed.str.replace(' rp', ' moneysymbol ')
        
    # Replace numbers with 'number'
    processed = processed.str.replace(r'\d+(\.\d+)?', ' number ')

    # Remove punctuation
    processed = processed.str.replace(r'[.,\/#!%\^&\*;:{}=\-_`~()?]', ' ')

    # Replace whitespace between terms with a single space
    processed = processed.str.replace(r'\s+', ' ')

    # Remove leading and trailing whitespace
    processed = processed.str.replace(r'^\s+|\s+?$', '')
    
    return processed

In [ ]:
word_features_f = open("word_features.pickle", "rb")
word_features = pickle.load(word_features_f)
word_features_f.close()
print("Created bag of word features!")

ordenc = OrdinalEncoder()
encoded_word_features = encoder.fit_transform(word_features)
for i in range(0, len(encoded_word_features)):
    encoded_word_features[i]+=4

# Filler Grammar to mark Start of a sentence and to add empty fillers to make equal shapes of data
word_features.append("<PAD>")
word_features.append("<START>")
word_features.append("<UNK>")
word_features.append("<UNUSED>")

encoded_word_features=np.append(encoded_word_features, [0,1,2,3])

print(word_features)
print(encoded_word_features)

In [ ]:
def find_features(message):
    words = word_tokenize(message)
    features = []
    features.append(1)
    for idx, word in enumerate(word_features):
        if word in words:
            features.append(encoded_word_features[idx])

    return features

In [ ]:
#Load Dataset

print("Reading data...")
df = pd.read_csv('corpus/sms_corpus/data.txt', engine='python', sep="<%>", header=None)
print("Data loaded")

classes = df[[0]]
sms_data = preproccess_df(df[1])

oneenc = OneHotEncoder(sparse=False)
Y = oneenc.fit_transform(classes)

# Now lets do it for all the messages
messages = list(zip(sms_data, Y))

# Call find_features function for each SMS message
featuresets = [(find_features(text), label) for (text, label) in messages]

preferred_range=25
for x in featuresets:
    arr_length=len(x[0])
    if arr_length < preferred_range:
        for i in range(arr_length, preferred_range):
            x[0].append(0)
    elif arr_length > preferred_range:
        diff=arr_length-preferred_range
        for i in range(0, diff):
            del x[0][i]

x_data, y_data = zip(*featuresets)
train_input, test_input, train_output, test_output = train_test_split(x_data, y_data, test_size = 0.2)

train_input=np.array(train_input)
test_input=np.array(test_input)
train_output=np.array(train_output)
test_output=np.array(test_output)

print(test_input)
print(train_output)

In [ ]:
vocab_size=2000
model = keras.Sequential()
model.add(keras.layers.Embedding(vocab_size, 16))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(3, activation="softmax"))

model.summary()

In [ ]:
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
history = model.fit(train_input,
                   train_output,
                   epochs=40,
                   batch_size=50,
                   validation_data=(test_input, test_output),
                   verbose=1)
model.save('sms_classifier_tf_model.h5')